# <center>Covid-19 in Italy</center>

#### <center> 29 March 2020 </center> 
#####  <center> A.P. </center> 



# Introduction 
With this exercise I tried to perform some consideration about data provided in "covid19-in-italy" database (only covid19_italy_region table). At first I have reported some considerations to remove redundant information. I have splitted data in a train and a test set. I have reported "time" and "space" analysis. Finally I have tried to predict "NewPositiveCases" with two supervised learning algorithms: Linear Regression and Random Forest. The performance measurement chosen is RMSE.
X train matrix is build with  Test Performed and Home Confinement data. These provision, which are the strongest weapons against virus.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.plotting import scatter_matrix
import sys
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
def load_data(data_path,data_fname):
    csv_path = os.path.join(data_path, data_fname)
    return pd.read_csv(csv_path)

In [ ]:
filename=filenames[0]
if (filenames[1].find('region')>0):
    filename=filenames[1]
    

In [ ]:
df_reg = load_data(dirname,filename)
df_reg.head()

In [ ]:
df_reg.shape

In [ ]:
df_reg.info()

In [ ]:
df_reg.describe(include='all')

"Country" column can be dropped because do not add any relevant information

In [ ]:
df_reg=df_reg.drop('Country',axis=1)

"SNo" can also be dropped cause it seems to be a Serial Number, counting numbers of row and therefore redundant w.r.t. dataframe index information 

In [ ]:
df_reg=df_reg.drop('SNo',axis=1)

In [ ]:
df_reg.columns

In [ ]:
df_reg.Date.value_counts()

Data related to time period from end of february to the end of march. It has to be converted to datetime object.

In [ ]:
df_reg.Date=pd.to_datetime(df_reg.Date)

In [ ]:
df_reg.info()

Last object type remained is "Region Name", which I suppose has been already coded by column "Region Code and therefore irrelevant. Let's see:

In [ ]:
df_reg_conv=df_reg.loc[:,['RegionCode','RegionName']]

In [ ]:
sum_devstd=0
for reg in df_reg_conv.RegionName.value_counts().index:
    print (reg)
    print ('Coded as:')
    print(str(df_reg_conv.loc[df_reg_conv.RegionName==reg].mean()[0]))
    sum_dev_std=+df_reg_conv.loc[df_reg_conv.RegionName==reg].std()[0]
print ('Total Dev Std:')
print(str(sum_dev_std))
    
    

Total sum of Standard Deviation is 0. Coding has been displayed before. Therefore "RegionName" column can be dropped.

In [ ]:
df_reg=df_reg.drop('RegionName',axis=1)

It is convenient to set the index to Date parameter:

In [ ]:
df_reg.set_index('Date')

As shown hereafter the increment of Total Positive Cases vs Time is exponential:

In [ ]:
df_reg.TotalPositiveCases.plot()

Train Test Split and Visual Analysis of Train Test Set:

In [ ]:
train_set,test_set=train_test_split(df_reg,test_size=0.2, random_state=10)

Visualizing Geographical Data:

In [ ]:
train_set.plot(kind='scatter',x='Longitude',y='Latitude')

It looks like Italy. Each dot is related to a Region. E.G. the dot in the middle lower part of the plot is Sicily:

In [ ]:
train_set.loc[df_reg_conv.RegionCode==19].loc[:,['Latitude','Longitude']].mean()

Now lets see 

In [ ]:
train_set.plot(kind="scatter", x="Longitude", y="Latitude", alpha=0.4,
    s=train_set['TotalPositiveCases'], figsize=(10,13),
    c='TestsPerformed', cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)
plt.legend()

The radius of each circle represent the Total Positive Cases and the color represents the Number of Test Performed.
The greatest number of test performed and positive cases are concentrated in Northern Italy

Looking for correlations

In [ ]:
corr_mx=train_set.iloc[:,1:].corr()

In [ ]:
col=train_set.iloc[:,1:].columns

In [ ]:
sns.set(font_scale=1.5)
hm=sns.heatmap(corr_mx,cbar=True,annot=True,square=True,fmt='.2f',annot_kws={'size':6},yticklabels=col,xticklabels=col)

The above heatmap shows uncorrelation between geographical data and medical ones. Therefore these data can be removed. 

In [ ]:
corr_mx=train_set.iloc[:,5:].corr()
col=train_set.iloc[:,5:].columns

In [ ]:
hm=sns.heatmap(corr_mx,cbar=True,annot=True,square=True,fmt='.2f',annot_kws={'size':9},yticklabels=col,xticklabels=col)

Hereafter the strongest correlations found (scc>0.9):
1. TotalHospitalizedPatients w.r.t. IntensiveCarePatients (scc=0.99)
2. TotalHospitalizedPatients w.r.t. CurrentPositiveCases (scc=0.99)
2. TotalHospitalizedPatients w.r.t. TotalPositiveCases (scc=0.98)
2. TotalHospitalizedPatients w.r.t. HomeConfinement (scc=0.93)
2. TotalHospitalizedPatients w.r.t. Recovered (scc=0.93)
2. TotalHospitalizedPatients w.r.t. Deaths (scc=0.95)
3. IntensiveCarePatients w.r.t. CurrentPositiveCases (scc=0.98)
4. IntensiveCarePatients w.r.t. HomeConfinement (scc=0.93)
4. IntensiveCarePatients w.r.t. Deaths (scc=0.91)
4. HomeConfinement w.r.t. Deaths (scc=0.92)
4. HomeConfinement w.r.t. TestsPerformed (scc=0.93)
4. Recovered w.r.t. CurrentPositiveCases (scc=0.92)
4. Recovered w.r.t. TotalPositiveCases (scc=0.96)
4. Recovered w.r.t. Deaths (scc=0.99)
4. Death w.r.t. CurrentPositiveCases (scc=0.95)
5. TotalPositiveCases w.r.t. Deaths (scc=0.98)

And hereafter represented the scatter matrix of these parameters:

In [ ]:
col=['TotalHospitalizedPatients','IntensiveCarePatients','CurrentPositiveCases','TotalPositiveCases','HomeConfinement','Recovered','Deaths','TestsPerformed']

In [ ]:
axes =scatter_matrix(train_set[col],figsize=(14,10))

for ax in axes.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha('right')

plt.tight_layout()
plt.gcf().subplots_adjust(wspace=0, hspace=0)
plt.show()

An interesting parameter to be predicted is the number of new positive cases.
A Strong 
I try to build a model 

In [ ]:
corr_mx.NewPositiveCases.sort_values(ascending=False)

To predict it I will use parameters with scc >0.8, with exception of Test Performed which, togheter with Home Confinement,are the strongest weapons against virus.

In [ ]:
train_set_m=train_set.iloc[:,4:]
test_set_m=test_set.iloc[:,4:]
train_set_m=train_set_m.drop('Deaths',axis=1)
train_set_m=train_set_m.drop('Recovered',axis=1)
test_set_m=test_set_m.drop('Deaths',axis=1)
test_set_m=test_set_m.drop('Recovered',axis=1)

In [ ]:
y_train_set_m=train_set_m.iloc[:,5]
y_test_set_m=test_set_m.iloc[:,5]
X_train_set_m=train_set_m.drop('NewPositiveCases',axis=1)
X_test_set_m=test_set_m.drop('NewPositiveCases',axis=1)

In [ ]:
X_train_set_m.describe()

Lets try to predict "NewPositiveCases" with two supervised learning algorithms: Linear Regression and Random Forest. The performance measurement chosen is RMSE.

Linear Regression:

In [ ]:
lin_reg=LinearRegression()
lin_reg.fit(X_train_set_m_std,y_train_set_m)

In [ ]:
y_test_predict=lin_reg.predict(X_test_set_m)
lin_rmse=np.sqrt(mean_squared_error(y_test_set_m,y_test_predict))
print(lin_rmse)

Random Forest Regressor:

In [ ]:
rnd_clf=RandomForestRegressor(n_estimators=30)
rnd_clf.fit(X_train_set_m,y_train_set_m)

In [ ]:
y_test_predict_rf=rnd_clf.predict(X_test_set_m)
lin_rmse_rf=np.sqrt(mean_squared_error(y_test_set_m,y_test_predict_rf))
print(lin_rmse_rf)

The best model is Linear Regression one (less RMSE)! Let's try to tune the Random Forest model and cross validate it.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [ ]:
param_grid=[{'n_estimators':[3,30,50,80,100]}]

In [ ]:
grid_search=GridSearchCV(rnd_clf,param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True)

In [ ]:
grid_search.fit(X_train_set_m,y_train_set_m)

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres=grid_search.cv_results_
for mean_score,params in zip(cvres["mean_test_score"],cvres["params"]):
    print(np.sqrt(-mean_score),params)

In [ ]:
rnd_clf=RandomForestRegressor(n_estimators=80)
rnd_clf.fit(X_train_set_m,y_train_set_m)
y_test_predict_rf=rnd_clf.predict(X_test_set_m)
lin_rmse_rf=np.sqrt(mean_squared_error(y_test_set_m,y_test_predict_rf))
print(lin_rmse_rf)

In [ ]:
from sklearn.model_selection import cross_val_score
scores=cross_val_score(rnd_clf,X_train_set_m,y_train_set_m,scoring="neg_mean_squared_error",cv=10)


In [ ]:
score_rmse=np.sqrt(-scores)
print(score_rmse)
print(score_rmse.mean())
print(score_rmse.std())

# Conclusion

The best model is still Linear Regression one with 4.68 of rmse vs 91.82 (MEAN) of Random Forest one.

*Thanks for the attention. Any feedback are  welcome.*